## 📥 Load and Prepare Data 

- This Notebook: Trains a **separate sliding-window forecasting model for each station** using **Facebook Prophet**
- This ensures that localized seasonality, trends, and anomalies are captured independently for each point.
- The notebook example shows training for one station — you can loop through all 242 to scale the forecast.
- Forecasts hourly temperature for **Jan–Feb 2025** and validates against actual ERA5 data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
# Change the path and file name
col_name = "Temperature"
# If you want to train for other points change the point
point = 118
df = pd.read_csv(f"path/file name_{point}.csv", index_col='Date', 
                  parse_dates=True,
                  engine='python',
                  usecols=['Date', col_name])
df = df.asfreq('h')
df.head()

In [ ]:
# Change Kelvin to Celcius
def kelvintodegc(kelvin):
    return kelvin - 273.15

In [ ]:
temp = df[[col_name]].copy()
temp['Temperature'] = temp['Temperature'].apply(kelvintodegc)
temp

In [ ]:
# Besause temperature is not normally distributed
# We will use quantile transformer to transform the data
from sklearn.preprocessing import QuantileTransformer
import numpy as np

# Example data: replace with your actual data array
X = np.array(temp[col_name]).reshape(-1, 1)

# Transform the data to follow a normal distribution
qt = QuantileTransformer(output_distribution='normal', random_state=0)
temp[col_name] = qt.fit_transform(X)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

plt.figure(figsize=(10, 5))

# Plot histogram of z-scores
plt.hist(temp[col_name], bins=100, alpha=0.5, label=col_name)

# Calculate parameters for the normal distribution
mu, sigma = temp[col_name].mean(), temp[col_name].std()

# Generate points along the x-axis range
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 1000)

# Calculate normal distribution scaled to match histogram
pdf = norm.pdf(x, mu, sigma)
scale_factor = len(temp[col_name]) * (xmax - xmin) / 100  # Number of points * bin width
plt.plot(x, pdf * scale_factor, 'r-', lw=2, label='Normal Distribution')

plt.legend(loc='best')
plt.show()

In [ ]:
# Lets create features for time series
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    return df

In [ ]:
# Here we will create lags for 720 hours and 8640 hours
def add_lags(df):
    target_map = df[col_name].to_dict()
    df['lag1'] = (df.index - pd.Timedelta('720 hours')).map(target_map)
    df['lag2'] = (df.index - pd.Timedelta('8640 hours')).map(target_map)
    #df['lag3'] = (df.index - pd.Timedelta('168 hours')).map(target_map)
    return df

In [ ]:
# Here we will create lags for 720 hours and 8640 hours for sliding window
def lag_1hr(df):
    df['lag1'] = df[col_name].shift(720)
    df = df.dropna()
    return df
def lag_7hr(df):
    df['lag2'] = df[col_name].shift(8640)
    df = df.dropna()
    return df

In [ ]:
temp = add_lags(temp)
temp = create_features(temp)
temp = temp.dropna()
temp.isna().sum()

In [ ]:
temp[col_name].max()

In [ ]:

# For sliding window we will create a future dataframe
# Create future dataframe
future_dates = pd.date_range(start='2025-01-01, 00:00:00', end='2025-12-31 00:00:00', freq='H')
future = pd.DataFrame(future_dates, columns=['ds'])
future['lag1']= np.nan  # Initialize lag1 with NaN values
future['lag2']= np.nan  # Initialize lag1 with NaN values
#future['lag3']= np.nan  # Initialize lag1 with NaN values
future[col_name]= np.nan  # Initialize lag1 with NaN values
future

In [ ]:
lastHour = temp[[col_name]].iloc[-1440:]
last7Hours = temp[[col_name]].iloc[-17280:]
#last21Hours = temp[[col_name]].iloc[-336:]


lastHour = lag_1hr(lastHour)
lastHour = lastHour.reset_index()
future['lag1'] = future["lag1"].fillna(lastHour["lag1"])

last7Hours = lag_7hr(last7Hours)
last7Hours = last7Hours.reset_index()
future['lag2'] = future["lag2"].fillna(last7Hours["lag2"])
#last21Hours = lag_21hr(last21Hours)
#last21Hours = last21Hours.reset_index()
#future['lag3'] = future["lag3"].fillna(last21Hours["lag3"])

future.head(20)

In [ ]:
future = future.set_index('ds')
future = create_features(future)

In [ ]:
future = future.reset_index()
future

In [ ]:
temp = temp.reset_index()
#temp = temp.drop(columns=['index']) # Run if you have index
temp = temp.rename(columns={'Date':'ds', col_name:'y'})
temp.head()

In [ ]:
temp.tail()

## 📈 Train Prophet Model 

In [ ]:
model_2 = Prophet(
        yearly_seasonality='auto',
        weekly_seasonality=True,
        daily_seasonality=True,
        changepoint_prior_scale=0.0005,
        seasonality_prior_scale=10.0, 
        stan_backend="CMDSTANPY"
        )
model_2.add_regressor('lag1')
model_2.add_regressor('lag2')

model_2.add_regressor('hour')
model_2.add_regressor('month')
model_2.add_regressor('year')
model_2.add_regressor('quarter')
model_2.add_regressor('dayofweek')
model_2.add_regressor('dayofmonth')
model_2.add_regressor("dayofyear")
model_2.fit(temp)

In [ ]:
future

## 🔁 Sliding Window Setup  

In [ ]:
# Sliding window
len_future = len(future)
iter = [i for i in range(len_future)]
for i in iter:
    
    if i>0 and i % 720 == 0:
        temp7 = future.iloc[i-720:i]
        for j in range(1,721):
            future['lag2'].iloc[i:i+j] = future['lag2'].iloc[i:i+j].fillna(temp7[col_name].iloc[j-1])

    #if i>0 and i % 8640 == 0:
        #temp21 = future.iloc[i-8640:i]
        #for p in range(0,8641):
            #future['lag3'].iloc[i:i+p] = future['lag3'].iloc[i:i+p].fillna(temp21[col_name].iloc[p-1])

    current_pred = future.iloc[i:i+1]
    y_pred = model_2.predict(current_pred)
    y_pred = y_pred.reset_index()
    future[col_name].iloc[i] = y_pred['yhat']
    temp1 = future.iloc[i]
    future['lag1'].iloc[i+1:i+2] = future['lag1'].iloc[i+1:i+2].fillna(temp1[col_name]) 
    print(i)    

In [ ]:
future.isna().sum()

In [ ]:
# Here we apply reverse transform to get the temperature back in Celsius
X = np.array(future[col_name]).reshape(-1, 1)

future[col_name] = qt.inverse_transform(X)

future

In [ ]:
future = future.dropna()
future

## 📊 Evaluate Predictions  

In [ ]:
# Change the path and file name
# Make sure you have subsetted the data for 2025 as well

dfor1 = pd.read_csv("2025/Pakistan_merged_data_points_2025_cleaned_118.csv")
#dfor = dfor1[[col_name]].apply(kelvintodegc).reset_index() # Run if you get index error
dfor1["Date"] = pd.to_datetime(dfor1["Date"])
dfor1.set_index('Date', inplace=True)
dfor = dfor1[[col_name]]
dfor["Temperature"] = dfor["Temperature"].apply(kelvintodegc)
dfor


In [ ]:
future['ds'] = pd.to_datetime(future['ds'])
future.set_index('ds', inplace=True)
future

In [ ]:

forcast_2025_model2_sub = future.iloc[:len(dfor)]
forcast_2025_model2_sub = forcast_2025_model2_sub[[col_name]]
forcast_2025_model2_sub.tail()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(dfor[col_name], label='Existing Data')
plt.plot(forcast_2025_model2_sub[col_name], label='Forecasted Data', alpha=0.5)
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(dfor['Temperature'], label='Existing Data')
plt.plot(forcast_2025_model2_sub['Temperature'], label='Forecasted Data', alpha=0.5)
plt.legend(loc='best')
plt.show()

In [ ]:
# The important metrics here is the MAE if its under 3 deggres it is good
# Note each points will have different MAE
# If any point has MAE > 3 then just chnage the lags form 1 month and 1 year to 1 week and 1 month or 1 day and 1 hour and so on
# Also chnage the lags in sliding window
# Do your experimentations and find the best combination of lags

print(mean_absolute_error(dfor[col_name], forcast_2025_model2_sub[col_name]))
print(r2_score(dfor[col_name], forcast_2025_model2_sub[col_name]))

In [ ]:
future.loc['2025-04-08 12:00:00']

In [ ]:
Temperature = future[['Temperature']].copy()
Temperature

In [ ]:
Temperature["t_min"] = Temperature["Temperature"].rolling(24).min()
Temperature["t_max"] = Temperature["Temperature"].rolling(24).max()
Temperature["t_mean"] = Temperature["Temperature"].rolling(24).mean()
Temperature['std'] = Temperature["Temperature"].rolling(24).std()
Temperature["ub"] = Temperature["t_mean"] + (Temperature["std"]*1.5)
Temperature["lb"] = Temperature["t_mean"] - (Temperature["std"]*1.5)
Temperature = Temperature.dropna()

In [ ]:
Temperature

In [ ]:
Temperature = Temperature.asfreq('1d')
Temperature

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(Temperature['t_mean'], label='Average Temp')
ax.plot(Temperature['t_min'], label='Min Temp', alpha=1)
ax.plot(Temperature['t_max'], label='Max Temp', alpha=1)
#ax.fill_between(Temperature.index, 0, Temperature['t_min'], color = 'orange', alpha=0.5)
#ax.fill_between(Temperature.index, Temperature['t_min'], Temperature['t_mean'], color = 'blue', alpha=0.6)
#ax.fill_between(Temperature.index, Temperature['t_mean'], Temperature['t_max'], color = 'green', alpha=0.5)
ax.set_title('Temperature for 2025', weight='bold')
ax.legend(loc='best')
fig.show()

## 💾 Save Forecast Output

Save the output if you want